# FENE beadspring with variable kuhn length experiment data analysis

In [15]:
import os

import polyflexmd.data_analysis.data.read as read
import polyflexmd.data_analysis.transform.transform as transform
import polyflexmd.data_analysis.plot.plot_system as plot_system
import polyflexmd.experiment_runner.config as config
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from pandarallel import pandarallel
import dask
import dask.dataframe
import multiprocessing


In [2]:
% load_ext autoreload
% autoreload 2

In [3]:
pandarallel.initialize(
    nb_workers=32,
    progress_bar=False,
    use_memory_fs=False
)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Read and process data

In [22]:
PATH_EXPERIMENT = "/scratch/ws/0/s4610340-bt-eea1-md-workspace/data/experiment_results/FENE-beadspring-vary-l_K/4-FENE-beadspring-vary-l_K/48267f61"

In [23]:
PATH_EXPERIMENT = pathlib.Path(PATH_EXPERIMENT)
PATH_SYSTEM_DEF = PATH_EXPERIMENT / "data/initial_system.data"
PATH_EC = PATH_EXPERIMENT / "4-FENE-beadspring-vary-l_K.toml"

### Read experiment config

In [24]:
conf = config.read_experiment_config(PATH_EC)
conf

ExperimentConfig(simulation_config=SimulationConfig(job=SlurmJobConfig(account='p_mdpolymer', time='64:00:00', partition='romeo', ntasks=12, cpus_per_task=32, mem_per_cpu=1000), simulation_model_path=PosixPath('simulations/FENE-beadspring-vary-l_K.lammps'), experiments_path=PosixPath('/scratch/ws/0/s4610340-bt-eea1-md-workspace/data/experiment_results'), variables={'kappa': [1.0, 4.0, 7.0, 10.0, 13.0, 20.0, 30.0, 40.0, 50.0]}), system_creator_config=SystemCreatorConfig(job=SlurmJobConfig(account='p_mdpolymer', time='1:00:00', partition='romeo', ntasks=1, cpus_per_task=4, mem_per_cpu=1000), venv_path=PosixPath('/scratch/ws/0/s4610340-bt-eea1-md-workspace/polyflexmd/.venv'), system_config=AnchoredFENEChainConfig(name='anchored-fene-chain', n_chains=200, n_monomers=128, monomer_type=2, bond_type=1, angle_type=1, bond_length=0.97, box_length=200, seed=77)), report_config=None)

In [25]:
kappas: list[float] = conf.simulation_config.variables["kappa"]
kappas

[1.0, 4.0, 7.0, 10.0, 13.0, 20.0, 30.0, 40.0, 50.0]

### Read initial system

In [26]:
system = read.read_lammps_system_data(PATH_SYSTEM_DEF)
system.atoms

,molecule-ID,type,x,y,z,ix,iy,iz
1,1,1,0.000000,0.000000,0.000000,0,0,0
2,1,1,0.764233,-0.425729,-0.419050,0,0,0
3,1,2,0.773821,-0.836732,0.459520,0,0,0
4,1,2,1.285179,-0.524169,-0.303184,0,0,0
5,1,2,0.842282,0.329744,-0.427985,0,0,0
...,...,...,...,...,...,...,...,...
25596,200,2,-2.657187,4.352830,12.311182,0,0,0
25597,200,2,-2.321878,4.135283,11.427360,0,0,0
25598,200,2,-2.115038,4.152888,10.479833,0,0,0
25599,200,2,-2.192981,5.112155,10.358871,0,0,0


In [27]:
system.bonds

,type,atom1,atom2
1,1,1,2
2,1,2,3
3,1,3,4
4,1,4,5
5,1,5,6
...,...,...,...
25396,1,25595,25596
25397,1,25596,25597
25398,1,25597,25598
25399,1,25598,25599


In [28]:
system.angles

,type,atom1,atom2,atom3
1,1,1,2,3
2,1,2,3,4
3,1,3,4,5
4,1,4,5,6
5,1,5,6,7
...,...,...,...,...
25196,1,25594,25595,25596
25197,1,25595,25596,25597
25198,1,25596,25597,25598
25199,1,25597,25598,25599


In [29]:
system.box

-100.000000 100.000000  xlo xhi
-100.000000 100.000000  ylo yhi
-100.000000 100.000000  zlo zhi

### Read and process trajectories

In [30]:
def read_and_process_kappas_df(experiment_path: pathlib.Path, kappas: list[float]):
    dfs = []
    for i in range(1, len(kappas) + 1):
        paths_trajectories = [
            experiment_path / "data" / f"polymer_relax-{i}.out",
            experiment_path / "data" / f"polymer-{i}.out"
        ]
        print(f"Reading {paths_trajectories}...")
        if not all(p.exists() for p in paths_trajectories):
            break

        df_trajectory_unfolded = transform.unfold_coordinates_df(
            trajectory_df=transform.join_raw_trajectory_df_with_system_data(
                raw_trajectory_df=read.read_multiple_raw_trajectory_dfs(paths_trajectories),
                system_data=system
            ),
            system_data=system
        )
        df_trajectory_unfolded["kappa"] = kappas[i]
        dfs.append(df_trajectory_unfolded)

    return pd.concat(dfs)


df_trajectory_unfolded_kappas = read_and_process_kappas_df(PATH_EXPERIMENT, kappas)
df_trajectory_unfolded_kappas

Reading [PosixPath('/scratch/ws/0/s4610340-bt-eea1-md-workspace/data/experiment_results/FENE-beadspring-vary-l_K/4-FENE-beadspring-vary-l_K/48267f61/data/polymer_relax-1.out'), PosixPath('/scratch/ws/0/s4610340-bt-eea1-md-workspace/data/experiment_results/FENE-beadspring-vary-l_K/4-FENE-beadspring-vary-l_K/48267f61/data/polymer-1.out')]...
Reading [PosixPath('/scratch/ws/0/s4610340-bt-eea1-md-workspace/data/experiment_results/FENE-beadspring-vary-l_K/4-FENE-beadspring-vary-l_K/48267f61/data/polymer_relax-2.out'), PosixPath('/scratch/ws/0/s4610340-bt-eea1-md-workspace/data/experiment_results/FENE-beadspring-vary-l_K/4-FENE-beadspring-vary-l_K/48267f61/data/polymer-2.out')]...



KeyboardInterrupt



In [ ]:
PATH_DATA_PROCESSED = PATH_EXPERIMENT / "data" / "processed"
PATH_DATA_PROCESSED.mkdir(exist_ok=True)

df_trajectory_unfolded_kappas.to_csv(PATH_DATA_PROCESSED / "trajectory_unfolded_kappas.csv", index=True)

## Calculate and plot observables

### End-to-End distance $\vec{R}$

In [ ]:
PATH_DATA_PROCESSED = PATH_EXPERIMENT / "data" / "processed"
PATH_DATA_PROCESSED.mkdir(exist_ok=True)

PATH_ETE = PATH_EXPERIMENT / "data" / "processed" / "ete.csv"

df_ete = df_trajectory_unfolded_kappas.groupby("kappa").apply(transform.calc_end_to_end_df)
df_ete.to_csv(PATH_ETE, index=True)